In [9]:
# ===========================
# Import Libraries
# ===========================

# Numerical & Array Operations
import numpy as np

# Deep Learning & Torch Utilities
import torch
import torch.nn as nn

import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, Subset, TensorDataset
import torchvision
from torchvision import datasets, transforms, models
from torchvision.datasets import ImageFolder
from torchvision.transforms import ToTensor

# Image Processing & Computer Vision
from PIL import Image
import albumentations as A
from albumentations.pytorch import ToTensorV2

# Machine Learning & Evaluation Metrics
from sklearn.preprocessing import normalize
from sklearn.metrics import accuracy_score
from sklearn.svm import LinearSVC

# Utilities
from pathlib import Path
from tqdm import tqdm

In [2]:
# Charger le fichier .npz
data = np.load("dataset_fine_tuning.npz", allow_pickle=True)

# Récupérer les ensembles de données
X_train = data['X_train']
y_train = data['y_train']
X_test = data['X_test']
y_test = data['y_test']

# Vérifier les dimensions (facultatif)
print(f"X_train: {len(X_train)}, y_train: {len(y_train)}")
print(f"X_test: {len(X_test)}, y_test: {len(y_test)}")


X_train: 80000, y_train: 80000
X_test: 20000, y_test: 20000


In [3]:
# Définir une classe Dataset personnalisée
class CustomDataset(Dataset):
    def __init__(self, X, y, transform=None):
        self.X = X
        self.y = y
        self.transform = transform

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        image = self.X[idx]
        image = np.array(image)
        image = np.transpose(image, (1, 2, 0))
        y = self.y[idx]
        if self.transform is not None:
            augmentations = self.transform(image=image)
            image = augmentations["image"]

        return image, y


In [31]:
# Charger ResNet-34 pré-entraîné
model = models.resnet34(pretrained=False)

# Chemin vers votre fichier de poids
weights_path = "resnet34_cerberus_torchvision .pth"

# Charger les poids sauvegardés
state_dict = torch.load(weights_path)
model.load_state_dict(state_dict)

/tmp/ipykernel_2423504/2224343637.py:8: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(weights_path)


<All keys matched successfully>

In [32]:
# Freeze all layers: disable gradient tracking
for p in model.parameters():
    p.requires_grad = False
# "Thaw" last layer (or whatever is relevant for you)
for p in model.layer1.parameters():
    p.requires_grad = True
for p in model.layer2.parameters():
    p.requires_grad = True
for p in model.layer3.parameters():
    p.requires_grad = True
for p in model.layer4.parameters():
    p.requires_grad = True
# Replace fully-connected part by some other classifier, e.g.

cnn_features = 512
num_classes = 6

model.fc =  nn.Sequential(
    nn.Linear(cnn_features, 100, bias=True),
    nn.ReLU(),
    nn.Linear(100, num_classes, bias=True),
)

In [33]:
device = torch.device("cuda:3" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
model=model.to(device)

Using device: cuda:3


In [5]:
def loss_accuracy(loss, Yhat, Y):


    L = loss(Yhat, Y)
    acc = (torch.sum(torch.argmax(Yhat, dim=1) == Y) / Y.size(0)) * 100


    return L, acc

In [14]:
# Évaluation en mini-batches, la RAM ne support pas le forward sur tout le test_loader
def evaluate_model(model, test_loader, loss_fn, device):
    model.eval()  # Mode évaluation
    total_loss = 0.0
    total_correct = 0
    total_samples = 0

    with torch.no_grad():  # Pas de calcul des gradients
        for X_batch, Y_batch in test_loader:
            X_batch, Y_batch = X_batch.float().to(device), Y_batch.to(device)  # Déplacer sur GPU
            Yhat = model(X_batch)  # Prédictions
            loss = loss_fn(Yhat, Y_batch)  # Calcul de la perte
            total_loss += loss.item() * X_batch.size(0)  # Ajouter perte pondérée par la taille du batch
            total_correct += (torch.argmax(Yhat, dim=1) == Y_batch).sum().item()  # Compter les bons résultats
            total_samples += X_batch.size(0)  # Total des échantillons
    model.train()
    avg_loss = total_loss / total_samples  # Moyenne des pertes
    avg_accuracy = (total_correct / total_samples) * 100  # Précision en pourcentage
    return avg_loss, avg_accuracy

# Dataset of 100K (Cerberus was trained on a subset of this dataset)

In [10]:
# Transformer X_train, X_test en tensors pour l'évaluation complète
X_train = torch.stack([x for x in X_train])
X_test = torch.stack([x for x in X_test])
y_train = torch.tensor(y_train)
y_test = torch.tensor(y_test)

In [26]:
nx=1000
nh_1=100
nh_2=100
ny=10
eta=0.1
n_iter=10
loss = torch.nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)
model.train()

# Planificateur StepLR : Réduction du LR tous les 10 epochs par un facteur 0.1
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)
batch_size=32

# epoch
for iteration in tqdm(range(n_iter), desc="Epochs", unit="epoch"):
    
        train_transform = A.Compose([
            A.ElasticTransform(alpha=1.0, sigma=50, p=0.25),
            A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=0.25),
            A.Rotate(limit=35, p=0.7),
            A.HorizontalFlip(p=0.5),
            A.VerticalFlip(p=0.1),
            A.GaussianBlur(blur_limit=(3, 7), p=0.2),
            ToTensorV2(),
        ])
        
        val_transforms = A.Compose(
        [ 
            ToTensorV2(),
        ],
        )
        
        # Recréer les datasets PyTorch
        train_dataset = CustomDataset(X_train, y_train, train_transform)
        test_dataset = CustomDataset(X_test, y_test, val_transforms)
        
        train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
        test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)
        
        
    
        for X_batch, Y_batch in tqdm(train_loader, desc="Batches", unit="batch", leave=False):
                X_batch, Y_batch = X_batch.to(device), Y_batch.to(device) 
                optimizer.zero_grad()
                Yhat = model(X_batch)  # Prédictions pour le batch actuel
                L, acc = loss_accuracy(loss, Yhat, Y_batch)  # Calcul de la perte et de la précision
                L.backward()
                optimizer.step()
        scheduler.step()
    
        
        Ltest, acctest = evaluate_model(model, test_loader, loss, device)
    
        title = "Iter {}:, acc test {:.1f}% ({:.2f})".format(iteration, acctest, Ltest)

        
    
        print(title)

Batches:   8%|█████▌                                                              | 203/2500 [00:20<03:55,  9.75batch/s]


Batches:  16%|██████████▊                                                         | 396/2500 [00:40<03:42,  9.44batch/s]


Batches:  23%|███████████████▉                                                    | 587/2500 [00:59<03:28,  9.18batch/s]


Batches:  29%|███████████████████▉                                                | 732/2500 [01:15<03:09,  9.32batch/s]


Batches:  35%|███████████████████████▊                                            | 874/2500 [01:31<02:53,  9.36batch/s]


...

Batches:  99%|██████████████████████████████████████████████████████████████████▏| 2468/2500 [04:27<00:03,  9.28batch/s]


Epochs:  20%|██████████████▍                                                         | 2/10 [10:02<40:04, 300.58s/epoch]

Iter 1:, acc test 10.3% (nan)



Epochs:  20%|██████████████▍                                                         | 2/10 [10:15<41:00, 307.54s/epoch]


KeyboardInterrupt: 

# EBHI-SEG Dataset

In [37]:
# Custom dataset to apply Albumentations transformations
class CustomDataset(Dataset):
    def __init__(self, dataset, transform=None):
        self.dataset = dataset
        self.transform = transform

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        image, label = self.dataset[idx]  # Image is a PIL Image
        image = np.array(image)  # Convert to NumPy array
        if self.transform:
            augmented = self.transform(image=image)
            image = augmented['image']
        return image, label
# Load the dataset
train_transform = A.Compose([
            A.ElasticTransform(alpha=1.0, sigma=50, p=0.25),
            A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=0.25),
            A.Rotate(limit=35, p=0.7),
            A.HorizontalFlip(p=0.5),
            A.VerticalFlip(p=0.1),
            A.GaussianBlur(blur_limit=(3, 7), p=0.2),
            ToTensorV2(),
        ])
dataset_path = "./EBHI-SEG-Class/"
base_dataset = ImageFolder(root=dataset_path)
custom_dataset = CustomDataset(base_dataset, transform=train_transform)

# Générer des indices aléatoires pour mélanger les données
indices = np.random.permutation(len(custom_dataset))

# Diviser en 80% entraînement et 20% test
train_indices = indices[:int(0.8 * len(indices))]
test_indices = indices[int(0.8 * len(indices)):]

# Créer des sous-ensembles
train_dataset = Subset(custom_dataset, train_indices)
test_dataset = Subset(custom_dataset, test_indices)

# Créer des DataLoaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [34]:
nx=1000
nh_1=100
nh_2=100
ny=10
eta=0.1
n_iter=30
loss = torch.nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)
model.train()

# Planificateur StepLR : Réduction du LR tous les 10 epochs par un facteur 0.1
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=15, gamma=0.1)
batch_size=32

# epoch
for iteration in tqdm(range(n_iter), desc="Epochs", unit="epoch"):
    
        train_transform = A.Compose([
            A.ElasticTransform(alpha=1.0, sigma=50, p=0.25),
            A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=0.25),
            A.Rotate(limit=35, p=0.7),
            A.HorizontalFlip(p=0.5),
            A.VerticalFlip(p=0.1),
            A.GaussianBlur(blur_limit=(3, 7), p=0.2),
            ToTensorV2(),
        ])
        
        val_transforms = A.Compose(
        [ 
            ToTensorV2(),
        ],
        )
        
        # Recréer les datasets PyTorch
        custom_dataset = CustomDataset(base_dataset, transform=train_transform)
    

        # Créer des sous-ensembles
        train_dataset = Subset(custom_dataset, train_indices)
        test_dataset = Subset(custom_dataset, test_indices)

        # Créer des DataLoaders
        train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
        test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)
        
        
    
        for X_batch, Y_batch in tqdm(train_loader, desc="Batches", unit="batch", leave=False):
                X_batch, Y_batch = X_batch.float().to(device), Y_batch.to(device) 
                optimizer.zero_grad()
                Yhat = model(X_batch)  # Prédictions pour le batch actuel
                L, acc = loss_accuracy(loss, Yhat, Y_batch)  # Calcul de la perte et de la précision
                L.backward()
                optimizer.step()
        scheduler.step()
    
        
        Ltest, acctest = evaluate_model(model, test_loader, loss, device)
    
        title = "Iter {}:, acc test {:.1f}% ({:.2f})".format(iteration, acctest, Ltest)

        
    
        print(title)

Epochs:   3%|██▍                                                                      | 1/30 [00:11<05:33, 11.51s/epoch]

Iter 0:, acc test 63.7% (1.09)



Epochs:   7%|████▊                                                                    | 2/30 [00:22<05:09, 11.05s/epoch]

Iter 1:, acc test 77.6% (0.69)



Epochs:  10%|███████▎                                                                 | 3/30 [00:32<04:53, 10.86s/epoch]

...

Iter 27:, acc test 81.2% (0.45)



Epochs:  97%|█████████████████████████████████████████████████████████████████████▌  | 29/30 [05:19<00:11, 11.07s/epoch]

Iter 28:, acc test 84.8% (0.39)



Epochs: 100%|████████████████████████████████████████████████████████████████████████| 30/30 [05:30<00:00, 11.01s/epoch]

Iter 29:, acc test 85.2% (0.40)


# SVM

In [50]:
# Custom dataset to apply Albumentations transformations
class CustomDataset(Dataset):
    def __init__(self, dataset, transform=None):
        self.dataset = dataset
        self.transform = transform

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        image, label = self.dataset[idx]  # Image is a PIL Image
        image = np.array(image)  # Convert to NumPy array
        if self.transform:
            augmented = self.transform(image=image)
            image = augmented['image']
        return image, label
# Load the dataset
train_transform = A.Compose([
            ToTensorV2(),
        ])
dataset_path = "./EBHI-SEG-Class/"
base_dataset = ImageFolder(root=dataset_path)
custom_dataset = CustomDataset(base_dataset, transform=train_transform)

# Générer des indices aléatoires pour mélanger les données
indices = np.random.permutation(len(custom_dataset))

# Diviser en 80% entraînement et 20% test
train_indices = indices[:int(0.8 * len(indices))]
test_indices = indices[int(0.8 * len(indices)):]

# Créer des sous-ensembles
train_dataset = Subset(custom_dataset, train_indices)
test_dataset = Subset(custom_dataset, test_indices)

# Créer des DataLoaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [51]:
# Charger ResNet-34 pré-entraîné
model = models.resnet34(pretrained=False)

# Chemin vers votre fichier de poids
weights_path = "resnet34_cerberus_torchvision .pth"

# Charger les poids sauvegardés
state_dict = torch.load(weights_path)
model.load_state_dict(state_dict)

/tmp/ipykernel_2423504/2224343637.py:8: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(weights_path)


<All keys matched successfully>

In [52]:
PRINT_INTERVAL = 50
def extract_features_Res(data, model):
    #####################
    ## Initialization   ##
    #####################
    # Initialize feature and label storage
    X = []
    y = []

    #####################
    ## Feature Extraction ##
    #####################
    for i, (input, target) in enumerate(data):
        # Print batch progress
        if i % PRINT_INTERVAL == 0:
            print('Batch {0:03d}/{1:03d}'.format(i, len(data)))



        # Extract features
        with torch.no_grad():  # Disable gradients for efficiency
            x = model(input.float())

        # Detach from computation graph and move to CPU if needed
        x = x.detach().cpu().numpy()
        target = target.cpu().numpy()

        # Append the features and labels
        X.append(x)
        y.append(target)

    # Concatenate all batches into single arrays and convert to tensor
    X = torch.tensor(np.concatenate(X, axis=0))  # X as a PyTorch tensor
    y = torch.tensor(np.concatenate(y, axis=0))  # y as a PyTorch tensor

    return X, y

In [53]:
model.eval()
    # Extraction des features
print('Feature extraction')
X_train, y_train = extract_features_Res(train_loader, model)
X_test, y_test = extract_features_Res(test_loader, model)

Feature extraction
Batch 000/056
Batch 050/056
Batch 000/014


In [54]:
print('Apprentissage des SVM')
accuracy = 0
svm = LinearSVC(C=1.0).fit(X_train, y_train)
y_hat = svm.predict(X_test)
y_test = y_test.cpu().numpy() if isinstance(y_test, torch.Tensor) else y_test
accuracy = accuracy_score(y_test, y_hat)
print(f"Accuracy = {round(accuracy*100,2)} %")

Apprentissage des SVM
Accuracy = 59.19 %
